# **Part II:  Homework 2**
## Course: Information Security
## **Autores:** Anahí Andrade & Mateo Salgado  
## **Fecha:** 23/11/2025  

## **1. Standard Digital Signature Protocol Review**


## **2. Vulnerability to Attack Vectors**


## **3. Research: Transport Layer Security (TLS 1.3)**




### 3.1 Arquitectura y evolución

El protocolo TLS 1.3 mantiene una estructura clara: un _handshake_ para negociación y autenticación, seguido de una _record layer_ que cifra los datos de aplicación.  
Con esta versión se simplificó mucho el proceso permitiendo que se reduzca la latencia (1-RTT para nuevos enlaces, opción de 0-RTT para reanudaciones) y cifrando más temprano en el flujo (IETF, 2018).  
Comparado con la versión previa siendo la TLS 1.2, el TLS 1.3 elimina el uso de intercambio RSA estático y de suites sin secreto perfecto hacia adelante o tambien PFS.  
Solo se admiten intercambios efímeros como (EC)DHE,  se suprimen la renegociación insegura y los modos de cifrado antiguos (CBC, RC4) (Assured AB, 2018).  
También se trabaja para que elementos como el SNI del _ClientHello_ queden protegidos mediante extensiones como **ECH** (Cloudflare, 2018).

---

### 3.2 Primitivas criptográficas empleadas

En TLS 1.3 se emplean varias primitivas clave:

- **Intercambio de claves:** (EC)DHE efímero, lo que garantiza secreto perfecto hacia adelante.  
- **Derivacion de claves:** se usa **HKDF** (“extract-then-expand”) para pasar desde el secreto primario al conjunto de claves de aplicación y reanudación (BearSSL, 2024).  
- **Cifrado autenticado:** todas las suites de cifrado son **AEAD**, típicamente AES-GCM y ChaCha20-Poly1305 (Cloudflare, 2018; Assured AB, 2018).  
- **Autenticación:** el servidor debe probar su identidad mediante certificado (X.509) y la operación _CertificateVerify_.  
  El cliente puede ser autenticado, aunque no siempre es obligatorio.

---

### 3.3 Cómo satisface confidencialidad, integridad y autenticación (C-I-A)

- **Confidencialidad:** todos los datos de aplicación viajan cifrados con AEAD y derivan de claves efímeras, garantizando que incluso si se compromete una clave futura no se puede descifrar el tráfico pasado (IETF, 2018).  
- **Integridad:** los modos AEAD incluyen autenticación de los datos y metadatos, y el mensaje _Finished_ asegura que el transcript del handshake no ha sido manipulado (IETF, 2018).  
- **Autenticación:** el servidor (y opcionalmente el cliente) presenta un certificado y demuestra la posesión de la clave privada.  
  La eliminación de estructuras inseguras anteriores reduce vectores de ataque históricos (EncryptionConsulting, 2024).

---

### 3.4 Aplicaciones modernas

TLS 1.3 es el estándar dominante en:

- **Navegación web como HTTPS y APIs**, donde los navegadores y CDNs lo adoptan por defecto para tener menor latencia y mayor seguridad (Cloudflare, 2018).  
- **Infraestructura de microservicios y bases de datos en transito**, donde se exige cifrado con PFS y AEAD para conexiones internas (EncryptionConsulting, 2024).  
- **Dispositivos móviles, IoT y redes de alto rendimiento**, donde se requiere tanto eficiencia como robustez criptográfica (Assured AB, 2018).  
- **Transporte con privacidad extendida:** la extensión **ECH** protege metadatos del cliente como el SNI en entornos que requieren máxima confidencialidad.



## **4. Design Problem**


### 4.1 Objetivo general

El Secure Electronic Contract Signing System o SECS como se menciona,  busca garantizar que dos partes Alice (Service Provider) y Bob (Client) puedan firmar electrónicamente un contrato crítico de forma segura, verificable y legalmente vinculante.
El diseño propuesto asegura dos propiedades esenciales de no repudio:

* Non-Repudiation of Origin (NRO): donde  ninguna parte puede negar haber firmado el contrato.

* Non-Repudiation of Submission/Receipt (NRS/NRR): donde  ninguna parte puede negar haber recibido el contrato firmado final.

### 4.2 Principios criptográficos utilizados

El sistema integra mecanismos modernos de criptografía asimétrica, hashing y sellado de tiempo, asegurando trazabilidad y validez legal:

|     Función                   | Mecanismo criptográfico                                                                                            |
| ------------------------- | ------------------------------------------------------------------------------------------------------------------ |
| Identidad y autenticación   | Certificados X.509 con RSA-PSS o ECDSA, emitidos por una Autoridad de Certificación confiable |
| Integridad del contrato       | SHA-256/SHA-512 como función hash                                                                              |
| Firma digital             | RSA-PSS / ECDSA firmando el hash del contrato                                                              |
| Sellado de tiempo    | Timestamp Authority (TSA) bajo RFC 3161, el cual garantiza prueba de existencia temporal                        |
| Cifrado en tránsito |   TLS 1.3 con AES-GCM o ChaCha20-Poly1305                                                              |
| Evidencia verificable       | Uso de OCSP stapling y cadena de certificados completa en cada firma                                           |


### 4.3 Arquitectura del sistema
La arquitectura posee tres capas principales:

1. Application Layer

Interfaz web o móvil donde Alice y Bob visualizan el contrato y aprueban la firma. Incluye controles de *What you see is what you sign*  o WYSIWYS para evitar manipulación visual del documento.

2. Security & Trust Layer

* Módulo de firma digital: genera el hash del documento y lo firma con la clave privada del usuario.

* TSA: añade sello de tiempo a la firma.

* Certificate Validation Module: verifica la validez y revocación del certificado mediante OCSP.

* Logging/Audit Service: registra cada operación ( ya se a firma, envío, recepción ) en un almacén inmutable (por ejemplo S3 con Object Lock o blockchain privada).

3. Communication Layer

Utiliza TLS 1.3 para garantizar cifrado y autenticacion mutua entre los nodos del sistema, evitando el famoso man in the middle  o Replay Attacks.

### 4.4 Flujo de protocolo propuesto

## **Obras de Referencia**



Assured AB. (2018). *T​LS 1.3 in a nutshell*. https://www.assured.se/posts/tls-1-3-nutshell  
BearSSL. (2024). *TLS 1.3 status — cryptographic algorithms*. https://www.bearssl.org/tls13.html  
Cloudflare. (2018). *A detailed look at RFC 8446 (a.k.a. TLS 1.3)*. *Cloudflare Blog*. https://blog.cloudflare.com/rfc-8446-aka-tls-1-3/  
EncryptionConsulting. (2024). *Understanding TLS 1.2 and TLS 1.3*. https://www.encryptionconsulting.com/tls-1-2-and-tls-1-3/  
Internet Engineering Task Force. (2018). *The Transport Layer Security (TLS) Protocol Version 1.3 (RFC 8446)*. https://datatracker.ietf.org/doc/html/rfc8446